# Download last.fm listening history

  - To see this analysis live, check out my article ["Analyzing Last.fm Listening History"](http://geoffboeing.com/2016/05/analyzing-lastfm-history/)
  - My last.fm page: http://www.last.fm/user/gboeing
  - API documentation: http://www.last.fm/api
  - For anything more complicated, you might use this Python wrapper for the API: https://github.com/pylast/pylast
  
This tool separately downloads your all-time most played tracks, artists, and albums. Then it downloads all of your scrobbles in order of recency. Each of these 4 data sets are saved to separate CSV files. It downloads the "all-time most" data separately because (at least for my data) my massive iTunes history scrobble-upload in 2007 is included in the all-time most played tracks/artists/albums but is excluded from the recent tracks API endpoint. For accurate analysis of my all-time scrobbles, I need to look at those separate all-time lists, or else 4 years of listening history (from iTunes) is ignored in the calculations.

Replace the "from keys import..." line of code with two new lines of code (replace placeholder values with your actual values):
  1. key=YOUR-LASTFM-API-KEY
  2. username=YOUR-LASTFM-USERNAME

In [2]:
import requests, json, time, pandas as pd

key='edfd8819fb5684d05911d64873c9f3d0'
username='shemer77'

In [3]:
# how long to pause between consecutive API requests
pause_duration = 0.2

## First get your all-time most played tracks, artists, and albums

In [4]:
url = 'https://ws.audioscrobbler.com/2.0/?method=user.get{}&user={}&api_key={}&limit={}&extended={}&page={}&format=json'
limit = 200 #api lets you retrieve up to 200 records per call
extended = 1 #api lets you retrieve extended data for each track, 0=no, 1=yes
page = 1 #page of results to start retrieving at

In [10]:
method = 'toptracks'
request_url = url.format(method, username, key, limit, extended, page)
artist_names = []
track_names = []
play_counts = []
response = requests.get(request_url).json()
for item in response[method]['track']:
    artist_names.append(item['artist']['name'])
    track_names.append(item['name'])
    play_counts.append(item['playcount'])

top_tracks = pd.DataFrame()
top_tracks['artist'] = artist_names
top_tracks['track'] = track_names
top_tracks['play_count'] = play_counts
top_tracks.to_csv('C:\\Users\\agc\\Documents\\GitHub\\data-visualization\lastfm-listening-history\\data\\lastfm_top_tracks.csv', index=None, encoding='utf-8')
top_tracks.head()

,artist,track,play_count
0,blink-182,Aliens Exist,39
1,Skillet,Monster,37
2,blink-182,Easy Target,34
3,Silverstein,Sacrifice,33
4,Silverstein,SOS,33


In [12]:
method = 'topartists'
request_url = url.format(method, username, key, limit, extended, page)
artist_names = []
play_counts = []
response = requests.get(request_url).json()
for item in response[method]['artist']:
    artist_names.append(item['name'])
    play_counts.append(item['playcount'])

top_artists = pd.DataFrame()
top_artists['artist'] = artist_names
top_artists['play_count'] = play_counts
top_artists.to_csv('C:\\Users\\agc\\Documents\\GitHub\\data-visualization\\lastfm-listening-history\\data\\lastfm_top_artists.csv', index=None, encoding='utf-8')
top_artists.head()

,artist,play_count
0,blink-182,590
1,Sum 41,497
2,Silverstein,495
3,Rise Against,479
4,Lil' Wayne,458


In [13]:
method = 'topalbums'
request_url = url.format(method, username, key, limit, extended, page)
artist_names = []
album_names = []
play_counts = []
response = requests.get(request_url).json()
for item in response[method]['album']:
    artist_names.append(item['artist']['name'])
    album_names.append(item['name'])
    play_counts.append(item['playcount'])

top_albums = pd.DataFrame()
top_albums['artist'] = artist_names
top_albums['album'] = album_names
top_albums['play_count'] = play_counts
top_albums.to_csv('C:\\Users\\agc\\Documents\\GitHub\\data-visualization\\lastfm-listening-history\\data\\lastfm_top_albums.csv', index=None, encoding='utf-8')
top_albums.head()

,artist,album,play_count
0,Silverstein,A Shipwreck in the Sand (Deluxe Edition),129
1,Silverstein,Rescue,124
2,Vinnie Paz,Season of the Assassin,115
3,Vinnie Paz,God of The Serengeti,109
4,Silverstein,This Is How the Wind Shifts,106


## Now get all your scrobbles

Last.fm provides this 'recenttracks' API method to get 'all' scrobbles. However, it seems to be pretty spotty for data from circa 2007. The best way to determine top tracks, artists, albums is with the cells above. However, the code below retrieves time series data of all scrobbles (but with the caveat of spotty data from 2007 and earlier).

Sample URL: https://ws.audioscrobbler.com/2.0/?method=user.getrecenttracks&user=gboeing&api_key={}&limit=1&extended=0&page=1&format=json

In [14]:
def get_scrobbles(method='recenttracks', username=username, key=key, limit=200, extended=0, page=1, pages=0):
    '''
    method: api method
    username/key: api credentials
    limit: api lets you retrieve up to 200 records per call
    extended: api lets you retrieve extended data for each track, 0=no, 1=yes
    page: page of results to start retrieving at
    pages: how many pages of results to retrieve. if 0, get as many as api can return.
    '''
    # initialize url and lists to contain response fields
    url = 'https://ws.audioscrobbler.com/2.0/?method=user.get{}&user={}&api_key={}&limit={}&extended={}&page={}&format=json'
    responses = []
    artist_names = []
    artist_mbids = []
    album_names = []
    album_mbids = []
    track_names = []
    track_mbids = []
    timestamps = []
    
    # make first request, just to get the total number of pages
    request_url = url.format(method, username, key, limit, extended, page)
    response = requests.get(request_url).json()
    total_pages = response[method]['@attr']['totalPages']
    if pages > 0:
        total_pages = min([total_pages, pages])
        
    print '{} total pages to retrieve'.format(total_pages)
    
    # request each page of data one at a time
    for page in range(1, int(total_pages) + 1, 1):
        if page % 10 == 0: print page,
        time.sleep(pause_duration)
        request_url = url.format(method, username, key, limit, extended, page)
        responses.append(requests.get(request_url))
    
    # parse the fields out of each scrobble in each page (aka response) of scrobbles
    for response in responses:
        scrobbles = response.json()
        for scrobble in scrobbles[method]['track']:
            # only retain completed scrobbles (aka, with timestamp and not 'now playing')
            if 'date' in scrobble.keys():
                artist_names.append(scrobble['artist']['#text'])
                artist_mbids.append(scrobble['artist']['mbid'])
                album_names.append(scrobble['album']['#text'])
                album_mbids.append(scrobble['album']['mbid'])
                track_names.append(scrobble['name'])
                track_mbids.append(scrobble['mbid'])
                timestamps.append(scrobble['date']['uts'])
                
    # create and populate a dataframe to contain the data
    df = pd.DataFrame()
    df['artist'] = artist_names
    df['artist_mbid'] = artist_mbids
    df['album'] = album_names
    df['album_mbid'] = album_mbids
    df['track'] = track_names
    df['track_mbid'] = track_mbids
    df['timestamp'] = timestamps
    df['datetime'] = pd.to_datetime(df['timestamp'].astype(int), unit='s')
    
    return df

In [15]:
# get all scrobbled tracks ever, in order of recency
scrobbles = get_scrobbles()

99 total pages to retrieve
10

 20 

30 

40 

50 

60 

70 

80 

90


In [17]:
# save the dataset
scrobbles.to_csv('C:\\Users\\agc\\Documents\\GitHub\\data-visualization\\lastfm-listening-history\\data\\lastfm_scrobbles.csv', index=None, encoding='utf-8')
print '{:,} total rows'.format(len(scrobbles))
scrobbles.head()

19,797 total rows


,artist,artist_mbid,album,album_mbid,track,track_mbid,timestamp,datetime
0,A Wilhelm Scream,657ec972-3ab1-48a8-b472-1f1c210aaf17,Career Suicide,bddcf7fa-6c03-41e0-85e5-3e14edd6299d,I Wipe My Ass With Showbiz,9b824a8b-1612-40cd-a713-a67799d2d065,1474592540,2016-09-23 01:02:20
1,Hollywood Undead,321fdfbb-426b-43f7-8295-fa9aca6348d9,Swan Songs,e15d287b-2688-3abe-9b24-b1cccf235467,Undead,1b8db0d4-d508-4bf0-9591-12d0c6a864d1,1474592276,2016-09-23 00:57:56
2,Bad Meets Evil,87f27b14-cbd0-44ea-93a3-6fead5cb45e7,Hell: The Sequel Deluxe Edition,,Fast Lane,ded07e2f-96c8-4c4f-bf0c-1cfedd2c8b40,1474592026,2016-09-23 00:53:46
3,Ace Hood,986ee464-cf8a-4e59-a71f-74756102067f,Ruthless,6e74c38f-f790-4eea-9e09-a6460b4ee2c7,Overtime (Ft Akon & T-Pain),,1474591700,2016-09-23 00:48:20
4,Royce Da 5′9″,a818c24f-d7c0-4420-afe9-09971652b87e,Layers,,Gottaknow,,1474591465,2016-09-23 00:44:25
